In [ ]:
%matplotlib inline
import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
import pyximport
pyximport.install(
    language_level=3,
    reload_support=True
)
from grating import *
import gc

In [ ]:
epsilon_Si = 13.491
epsilon_SiO2 = 2.085136
n_freqs = 200
freqs = np.linspace(0.15, 0.25, n_freqs)
n_grating_layers = 15

np.random.seed(42)
D = np.random.random_sample(n_grating_layers)
gr = Grating(epsilon_Si, epsilon_SiO2, D)
ps = gr.propagator(freqs)
ts = list(map(operator_t, ps))
plt.plot(freqs, [np.abs(t[0, 0]) ** 2 for t in ts])
plt.ylim(0, 1)

In [ ]:
# Benchmarking
%timeit propagator_layer(0., epsilon_Si, 2)
%timeit gr.propagator(0.2)
%timeit gr.propagator(freqs)

In [ ]:
batch_size = 1000
n_grating_layers = 15

a = 1
freqs = np.linspace(0.15 / a, 0.25 / a, n_freqs)


from produce_data import save_samples
fname = 'dataset.npz'
samples = dict(D=[], R=[])
try:
    i = 0
    with tqdm(total=batch_size, leave=False) as pbar:
        while True:
            D = np.random.random_sample(n_grating_layers)
            gr = Grating(epsilon_Si, epsilon_SiO2, D)
            ps = gr.propagator(freqs)
            ts = np.array(list(map(operator_t, ps)))
            i += 1
            pbar.update(1)
            samples['D'].append(D)
            samples['R'].append(np.abs(ts[:, 0, 0]) ** 2)
            if i == batch_size:
                save_samples(fname, samples)
                samples = dict(D=[], R=[])
                gc.collect()
                pbar.reset()
                i = 0
except KeyboardInterrupt:
    save_samples(fname, samples)